## Fetch Spot Data 

In [1]:
import os
import ccxt
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import time
import natsort
import glob

import sys
from pathlib import Path

# Add the parent directory to the system path
parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))

from utils.general import check_missing_timestamps
from utils.general import OHLCVScraper, get_top_symbol_by_volume

# Load environment variables
load_dotenv()

# Print the ccxt library version
print(f"ccxt version: {ccxt.__version__}")

ccxt version: 4.4.40


In [2]:
API_KEY = os.getenv('BITGET_API_KEY')
SECRET_KEY = os.getenv('BITGET_SECRET_KEY')
PASSWORD = os.getenv('BITGET_PASSWORD')
MARKET_TYPE = "spot"
EXCHANGE_ID = "bitget"
PATH_SAVE = f"/home/ubuntu/project/finance/cex-market-analysis/src/data/{EXCHANGE_ID}/{MARKET_TYPE}"
TIMEFRAME = '1m'
SYMBOL = "XRP/USDT"
START_DATE_STR = "2024-12-28 00:00:00"
END_DATE_STR = "2025-01-01 00:00:00"

exchange = getattr(ccxt, EXCHANGE_ID)({
'apiKey': API_KEY,
'secret': SECRET_KEY,
'password': PASSWORD,
'options': {
    'defaultType': MARKET_TYPE},
    'enableRateLimit': True
})

In [3]:
df_symbols = get_top_symbol_by_volume(exchange=exchange, pair_filter="/USDT", top_n=100)
df_symbols.head()

,symbol,volume_24h,price
14,ETH/USDT,5.400651e+08,3585.4000
13,BTC/USDT,4.456413e+08,97693.0100
51,BGB/USDT,2.763295e+08,6.1380
6,DOGE/USDT,2.581414e+08,0.3865
764,BIO/USDT,2.025999e+08,0.7450


In [ ]:
from utils.general import OHLCVScraper

scraper = OHLCVScraper(path_save=PATH_SAVE, exchange=exchange, exchange_id=EXCHANGE_ID)
scraper.scrape_candles_to_csv(
                symbol=SYMBOL,
                timeframe=TIMEFRAME,
                start_date_str=START_DATE_STR,
                end_date_str=END_DATE_STR,
                limit=100)

In [ ]:
df_spot = pd.read_csv("/home/ubuntu/project/finance/cex-market-analysis/src/data/bitget/spot/bitget/XRP_USDT_1m.csv")
df_spot['date'] = pd.to_datetime(df_spot['date'])
df_spot.set_index('date', inplace=True)
df_spot.head()

In [21]:
missing = check_missing_timestamps(df_spot, freq='1min')

In [ ]:
df_spot = df_spot.resample('4h').agg({
    'open': 'first',     # First price in the 1-hour window (Open)
    'high': 'max',       # Maximum price in the 1-hour window (High)
    'low': 'min',        # Minimum price in the 1-hour window (Low)
    'close': 'last',     # Last price in the 1-hour window (Close)
    'volume': 'sum'      # Total volume in the 1-hour window
})
df_spot.head()

## Fetch data from FUTURE

In [5]:
API_KEY = os.getenv('BITGET_API_KEY')
SECRET_KEY = os.getenv('BITGET_SECRET_KEY')
PASSWORD = os.getenv('BITGET_PASSWORD')
MARKET_TYPE = "future"
EXCHANGE_ID = "bitget"
PATH_SAVE = f"/home/ubuntu/project/finance/cex-market-analysis/src/data/{EXCHANGE_ID}/{MARKET_TYPE}"
TIMEFRAME = '1m'
SYMBOL = "XRP/USDT:USDT"
START_DATE_STR = "2024-12-28 00:00:00"
END_DATE_STR = "2025-01-01 00:00:00"

exchange = getattr(ccxt, EXCHANGE_ID)({
'apiKey': API_KEY,
'secret': SECRET_KEY,
'password': PASSWORD,
'options': {
    'defaultType': MARKET_TYPE},
    'enableRateLimit': True
})

In [18]:
df_symbols = get_top_symbol_by_volume(exchange=exchange, pair_filter="/USDT:USDT", top_n=100)
df_symbols = df_symbols.reset_index(drop=True)
df_symbols["symbol"].values


array(['BTC/USDT:USDT', 'ETH/USDT:USDT', 'XRP/USDT:USDT', 'SOL/USDT:USDT',
       'DOGE/USDT:USDT', 'MOCA/USDT:USDT', 'PEPE/USDT:USDT',
       'ADA/USDT:USDT', 'SUI/USDT:USDT', 'BIO/USDT:USDT', 'XLM/USDT:USDT',
       'UNI/USDT:USDT', 'HBAR/USDT:USDT', 'FARTCOIN/USDT:USDT',
       'BGB/USDT:USDT', 'BRETT/USDT:USDT', 'CHILLGUY/USDT:USDT',
       'AGLD/USDT:USDT', 'LINK/USDT:USDT', 'ENA/USDT:USDT',
       'ONDO/USDT:USDT', 'VIRTUAL/USDT:USDT', 'NEIROETH/USDT:USDT',
       'PNUT/USDT:USDT', 'UXLINK/USDT:USDT', 'SHIB/USDT:USDT',
       'WIF/USDT:USDT', 'DOT/USDT:USDT', 'TROY/USDT:USDT', 'IO/USDT:USDT',
       'STEEM/USDT:USDT', 'AI16Z/USDT:USDT', 'AIXBT/USDT:USDT',
       'SPX/USDT:USDT', 'AVAX/USDT:USDT', 'GIGA/USDT:USDT',
       'MOVE/USDT:USDT', 'SAND/USDT:USDT', 'PHA/USDT:USDT',
       'DegenReborn/USDT:USDT', 'TAO/USDT:USDT', 'AAVE/USDT:USDT',
       'FET/USDT:USDT', 'NEAR/USDT:USDT', 'GOAT/USDT:USDT',
       'NOT/USDT:USDT', 'WLD/USDT:USDT', 'USUAL/USDT:USDT',
       'LTC/USDT:USDT',

In [14]:
from utils.general import OHLCVScraper

scraper = OHLCVScraper(path_save=PATH_SAVE, exchange=exchange, exchange_id=EXCHANGE_ID)
scraper.scrape_candles_to_csv(
                symbol=SYMBOL,
                timeframe=TIMEFRAME,
                start_date_str=START_DATE_STR,
                end_date_str=END_DATE_STR,
                limit=100)

Fetched data up to 2024-12-28 01:40:00.001000
Fetched data up to 2024-12-28 03:20:00.001000
Fetched data up to 2024-12-28 05:00:00.001000
Fetched data up to 2024-12-28 06:40:00.001000
Fetched data up to 2024-12-28 08:20:00.001000
Fetched data up to 2024-12-28 10:00:00.001000


KeyboardInterrupt: 

In [26]:
df_future = pd.read_csv("/home/ubuntu/project/finance/cex-market-analysis/src/data/bitget/future/bitget/XRP_USDT:USDT_1m.csv")
df_future['date'] = pd.to_datetime(df_future['date'])
df_future.set_index('date', inplace=True)
missing = check_missing_timestamps(df_future, freq='1min')

In [ ]:
df_future = df_future.resample('4h').agg({
    'open': 'first',     # First price in the 1-hour window (Open)
    'high': 'max',       # Maximum price in the 1-hour window (High)
    'low': 'min',        # Minimum price in the 1-hour window (Low)
    'close': 'last',     # Last price in the 1-hour window (Close)
    'volume': 'sum'      # Total volume in the 1-hour window
})
df_future.head()

In [ ]:
df_future - df_spot

In [17]:
PATH_SAVE = "/home/ubuntu/project/finance/cex-market-analysis/src/data/bitget"
files_path = natsort.natsorted(glob.glob(os.path.join(PATH_SAVE, "*.csv"), recursive=False))
for file in files_path:
    df = pd.read_csv(file)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    missing = check_missing_timestamps(df, freq='1min')
    if not missing.empty:
        print("Missing timestamps:")
        print(file)